# BYTE-PAIR-ENCODING del Quijote
--- 
Este es una forma simple de compresión de datos en la que el par más común de bytes consecutivos de datos se reemplaza con un byte que no ocurre dentro de esos datos. Aquí, el objetivo no es la compresión de datos, sino la codificación de texto en un idioma dado como una secuencia de 'tokens', utilizando un vocabulario fijo de diferentes tokens. La mayoría de las palabras se codificarán como un solo token, mientras que las palabras raras se codificarán como una secuencia de unos pocos tokens, donde estos tokens representan partes de palabras significativas.

### Equipo cangrejo      
* Montaño Preciado Alondra Karolina
* Velasquez Hidalgo Luis Juventino
* Navarro Lopez Malcom Hiram
* Faz Leal Juan Carlos


### Fuentes
- Medium con la informacion: https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0

- Codigo en el cual nos estamos inspirando: https://leimao.github.io/blog/Byte-Pair-Encoding/


## **Funciones y utilidades usadas en la implementacion**

In [55]:
import re, collections

### `Clean_text`
___
Se usara esta funcion para tratar el texto y borrar tanto los numeros como cualquier caracter extraño. 
Se decidio que nos quedaremos solamente con las palabras. 

In [56]:
caracteres_especiales = r"[^\w\s]"
numeros = r'\d'
caracteres_y_numeros = r'[^\w\s]|\d'

def clean_text(text, clean_type='all'):
    if clean_type == 'commas':
        text = re.sub(caracteres_especiales, '', text)
    elif clean_type == 'numbers':
        text = re.sub(numeros, '', text)
    else:
        text = re.sub(caracteres_y_numeros, '', text)
    return text

##### ***Ejemplo de uso ...***

In [57]:
print(clean_text("*** START OF THIS PROJECT GUTENBERG EBOOK DON QUIJOTE ***","commas"))
print(clean_text("Posting Date: April 27, 2010 [EBook #2000]"))
print(clean_text("YO, EL REY."))
print(clean_text("-Por Dios, hermano, vuestras aciones."))
print(clean_text("»En lo que toca el poner anotaciones al fin del libro,"))
print(clean_text("''¿Para qué conmigo flo-?''"))

 START OF THIS PROJECT GUTENBERG EBOOK DON QUIJOTE 
Posting Date April   EBook 
YO EL REY
Por Dios hermano vuestras aciones
En lo que toca el poner anotaciones al fin del libro
Para qué conmigo flo


### `get_vocab()`
___
La función get_vocab lee un archivo de texto (especificado por filename) y devuelve un diccionario de frecuencias de palabras (vocabulario).
Cada línea en el archivo de texto se divide en palabras y cada palabra se agrega al vocabulario con un contador de frecuencia inicial de 1.
Si la palabra ya existe en el vocabulario, su contador se incrementa en 1.

- filename: Es un argumento de entrada para la función, es el nombre (o la ruta) del archivo de texto que se va a leer para crear el vocabulario.
- cleaning_function: Es una funcion que nos permite limpiar el texto.
- return: Es una instrucción que se utiliza para devolver un valor desde la funcion, en este caso, la función get_vocab devuelve el diccionario de frecuencias
  de palabras (vocabulario) creado en el cuerpo de la función, que se almacena en la variable vocab.


In [58]:
def get_vocab(filename , cleaning_function):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            line = cleaning_function(line)
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + '</w>'] += 1
    return vocab

### `get_stats()`
___
La función get_stats toma un vocabulario, el cual tiene la forma de un diccionario, y retorna otro diccionario con los pares de simbolos de nuestro vocabulario y la frecuencia con la que estos pares aparecen. Si el par ya existe en el diccionario, su contador se incrementa en 1.

In [59]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

### `merge_vocab()`
___
Funcion que junta los dos tokens que mas aparecen consecutivamente en el vocabulatio.
El bigrama es la union de los dos tokens que mas aparecen. 

Si la palabra contiene el bigrama, "w_out" obtiene el valor de la palabra actual remplazando los dos tokens por el bigrama, de lo contrario "w_out" obtiene conserva el valor de la palabra de 

la instruccion:
`v_out[w_out] = v_in[word]`
nos asegura que la frecuencia de la *posible* nueva palabra se conserva.
- pari: El par de tokens que mas se repiten  
- v_in: El vocabulario actual de palabras.
- return: El nuevo vocabulario, donde la ocurrencia del par de tokens 'pair' se remplazo por su union.

In [60]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

### `get_tokens_from_vocab()`
___
Es una función que produce dos objetos a partir de un vocabulario. Uno de los objetos contendra un diccionario que asocia cada símbolo individual en el vocabulario a la suma de las frecuencias de todas las palabras que contienen ese token. El otro objeto también es un diccionario pero este asocia acada palabra del vocabulario una lista de tokens individuales.

como un diccionario predeterminado y luego recorre cada entrada en vocabulario. Para cada entrada, la función divide la palabra en tokens individuales y agrega la frecuencia de la palabra a la frecuencia total de cada token. Además, la función agrega una entrada al diccionario que asocia la palabra completa con su lista de tokens. Finalmente, la función devuelve ambos diccionarios.

In [61]:
def get_tokens_from_vocab(vocab):
    # se crea un diccionario de frecuencias y de tokenizacion de tokens vacio
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    # se recorre el vocabulario de palabras y frecuencias de palabras del corpus
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
        # se devuelve el diccionario de frecuencias de tokens y el diccionario de tokenizacion de tokens
    return tokens_frequencies, vocab_tokenization


### `measure_token_length()`
___

La función measure_token_lengt mide la longitud de un token de texto. Si el token termina en \</w\>, se devuelve
la longitud de la palabra sin el sufijo \</w\> más 1. 
Si el token no termina en \</w\>, se devuelve la longitud del token
tal como está. Esto significa que la función devuelve la longitud de la palabra real o la longitud de la palabra con el sufijo \</w\> contado como 1.

-token: Es un argumento de entrada para la función measure_token_length. Es una cadena de texto que
representa una palabra o una secuencia de caracteres. 

La función mide su longitud y devuelve el resultado.
-return len(token): Es una instrucción que devuelve la longitud de la cadena de texto token. La función len en
python devuelve la cantidad de caracteres en una cadena de texto.
En este caso, si token no termina en \</w\>, se devuelve su longitud usando return len(token). De lo contrario,
se devuelve len(token[:-4]) + 1, lo que representa la longitud de token sin los últimos 4
caracteres (que son \</w\>) más 1.

In [62]:
def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

### `tokenize_word()`
___
Es una función de tokenización de palabras basada en el algoritmo de búsqueda de coincidencia de una lista de tokens conocidos. Toma como entrada una cadena de texto y una lista de tokens conocidos ordenados. Si la cadena de texto y la lista de tokens conocidos no son vacías, se itera sobre la lista de tokens conocidos en orden y se buscan coincidencias con la cadena de texto

In [63]:
### Funcion para tokenizar una palabra con un vocabulario de tokens ordenados por frecuencia y longitud de token ###
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    
    if sorted_tokens == []:
        return [unknown_token]
    string_tokens = []
    # se recorre el vocabulario de tokens ordenados por frecuencia y longitud de token #
    for i in range(len(sorted_tokens)):
    
        token = sorted_tokens[i]
        
        # se reemplaza el punto por [.] para que no sea interpretado como cualquier caracter #
        token_reg = re.escape(token.replace('.', '[.]'))
        
        # se busca el token en la palabra y se obtienen las posiciones de inicio y fin de cada ocurrencia #
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        
        #si no se encuentra el token en la palabra se continua con el siguiente token #
        if len(matched_positions) == 0:
            continue
            # si se encuentra el token en la palabra se obtienen las posiciones de inicio y fin de cada ocurrencia #
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        # se recorre la palabra desde el inicio hasta la posicion de inicio de cada ocurrencia del token #
        substring_start_position = 0
        
        for substring_end_position in substring_end_positions:
            # se obtiene la subcadena desde la posicion de inicio de la palabra hasta la posicion de inicio de la ocurrencia del token #
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            #se actualiza la posicion de inicio de la palabra para continuar desde la posicion de fin de la ocurrencia del token #
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        # se tokeniza la subcadena restante de la palabra #
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    # se devuelve la lista de tokens de la palabra #
    return string_tokens

## **Implementacion usando el Quijote**
---


Aqui creamos la funcion de Byte_pair.
La funcion usa iterativamente las funciones definidas anteriormente.

La variable verbose se utiliza para especificar que tantos mensajes devuelve la funcion.


In [64]:
def Byte_pair_encoder(vocab , num_merges=10 ,verbose=0):
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    if(verbose >= 2):
        print('_-_-_-_-_-_-_-')
        print('Tokens Before BPE')
        print('All tokens: {}'.format(tokens_frequencies.keys()))
        print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
        print('_-_-_-_-_-_-_-')
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)
        tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)

        if(verbose >= 1):
            print('_-_-_-_-_-_-_-')
            print('Iter: {}'.format(i))
            print('Best pair: {}'.format(best))
            print('All tokens: {}'.format(tokens_frequencies.keys()))
            print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    return tokens_frequencies,vocab_tokenization
    


#### **Vamos a ver como se comporta con el primer capitulo del quijote!!!**

In [67]:
""" 
Leemos solo el primer capitulo del quijote
Nos quedamos solo con las palabras 
"""
vocab = get_vocab('primer_capitulo.txt',clean_text)    
Byte_pair_encoder(vocab , num_merges= 5,verbose=2) 

_-_-_-_-_-_-_-
Tokens Before BPE
All tokens: dict_keys(['T', 'h', 'e</w>', 'P', 'r', 'o', 'j', 'e', 'c', 't</w>', 'G', 'u', 't', 'n', 'b', 'g</w>', 'E', 'B', 'k</w>', 'f</w>', 'D', 'n</w>', 'Q', 'i', 'y</w>', 'M', 'g', 'l</w>', 'd', 'C', 'v', 'a', 's</w>', 'S', 'a</w>', 'f', 'r</w>', 's', 'y', 'w', 'o</w>', 'd</w>', 'h</w>', 'l', 'm', 'Y', 'u</w>', 'p', 'L', 'A', 'R', 'T</w>', 'O', 'F</w>', 'H', 'I', 'S</w>', 'J', 'U', 'N', 'G</w>', 'K</w>', 'N</w>', 'E</w>', 'x', 'w</w>', 'L</w>', 'q', 'A</w>', 'á', 'ñ', 'é', 'í', 'Y</w>', 'i</w>', 'V', 'O</w>', 'ó', 'á</w>', 'F', 'ú', 'z', 'z</w>', 'ó</w>', 'í</w>', 'é</w>', 'É', 'R</w>', 'Ó', 'C</w>', 'X', 'Z', 'ü', 'm</w>', 'c</w>', 'x</w>', 'Z</w>', 'Á', 'ú</w>', 'Í'])
Number of tokens: 100
_-_-_-_-_-_-_-
_-_-_-_-_-_-_-
Iter: 0
Best pair: ('q', 'u')
All tokens: dict_keys(['T', 'h', 'e</w>', 'P', 'r', 'o', 'j', 'e', 'c', 't</w>', 'G', 'u', 't', 'n', 'b', 'g</w>', 'E', 'B', 'k</w>', 'f</w>', 'D', 'n</w>', 'Q', 'i', 'y</w>', 'M', 'g', 'l</w>', 'd', '

(defaultdict(int,
             {'T': 25,
              'h': 182,
              'e</w>': 321,
              'P': 28,
              'r': 886,
              'o': 661,
              'j': 75,
              'e': 1258,
              'c': 632,
              't</w>': 15,
              'G': 13,
              'u': 455,
              't': 702,
              'en': 234,
              'b': 219,
              'g</w>': 5,
              'E': 42,
              'B': 14,
              'k</w>': 4,
              'f</w>': 3,
              'D': 37,
              'n</w>': 369,
              'Q': 16,
              'i': 956,
              'y</w>': 183,
              'M': 25,
              'g': 202,
              'l</w>': 226,
              'de</w>': 246,
              'C': 33,
              'v': 170,
              'a': 1171,
              'n': 538,
              's</w>': 378,
              'S': 20,
              'd': 551,
              'a</w>': 563,
              'f': 108,
              'r</w>': 192,
            

#### **Vamos a poner a prueba los decoders y ver como se comporta con palabras nunca antes vistas !!**

In [71]:
#Separamos esta linea para no tener que correrla siempre
tokens_frequencies,vocab_tokenization = Byte_pair_encoder(vocab , num_merges=1000) 

In [70]:
# Let's check how tokenization will be for a known word
word_given_known = 'medio</w>'
word_given_unknown = 'Ilikeeatingapples!</w>'

""" 
Ordena el diccionario tokens_frequencies usando la combinacion de dos criterios:
    1. La llamada a la funcion measure_token_length
    2. La frecuencia de los tokens.
    Los tokens con una longitud mayor en el resultado de measure_token_length serán
    primero en la lista, y si hay varios tokens con la misma longitud, entonces aquellos
    con una frecuencia más alta serán primero en la lista.
"""
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)

""" 
Es una lista de los tokens ordenados en función de la longitud de los tokens y sus frecuencias
    donde los tokens con mayor longitud y mayor frecuencia se encuentran en la parte superior de la lista.
"""
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]
print(sorted_tokens)


word_given = word_given_known 
print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

['entendimiento</w>', 'pensamientos</w>', 'anotaciones</w>', 'Aristóteles</w>', 'caballerías</w>', 'seiscientos</w>', 'seguramente</w>', 'sentencias</w>', 'tratáredes</w>', 'intitulado</w>', 'imprimiere</w>', 'Excelencia</w>', 'Valladolid</w>', 'hubiéredes</w>', 'principio</w>', 'ingenioso</w>', 'Cervantes</w>', 'maravedís</w>', 'impresión</w>', 'Gutenberg</w>', 'caballero</w>', 'Escritura</w>', 'anotación</w>', 'compuesto</w>', 'Francisco</w>', 'erudición</w>', 'epigramas</w>', 'historia</w>', 'original</w>', 'imprimir</w>', 'Saavedra</w>', 'licencia</w>', 'servicio</w>', 'discreto</w>', 'márgenes</w>', 'facultad</w>', 'nuestros</w>', 'traigáis</w>', 'conforme</w>', 'mandamos</w>', 'poniendo</w>', 'quisiera</w>', 'contento</w>', 'catálogo</w>', 'taciones</w>', 'Quijote</w>', 'vuestra</w>', 'nuestro</w>', 'vuestro</w>', 'nuestra</w>', 'autores</w>', 'hidalgo</w>', 'Consejo</w>', 'trabajo</w>', 'persona</w>', 'sonetos</w>', 'Project</w>', 'QUIJOTE</w>', 'Andrada</w>', 'primero</w>', 'hi

# Conclusiones
___
Me diverti mucho. El algoritmo es elegante pues con pocas lineas de codigo logra cosas sorprendentes. Esto en gran parte a las expresiones regulares, pues con una sola instruccion hace operaciones que conlleban bastante trabajo. 

El alcance de cada funcion esta bien definido y se puede entender lo que hace por medio de su nombre.

Aunque el texto utilizado fue el Quijote, hay que reconocerle a este texto la capacidad que tiene de encontrar patrones indistintibamente del lenjuage, pues funciona a partir de caracteres y *frecuencias*. 

Aun asi, el paso de añadir el token \<\w\> al final de cada palabra reafirma el pensamiento de que siempre se necesitara la intervencion del humano aunque sea para hacer una clasificacion minima de las palabras.    
